In [1]:
#From Step #2:
vqa_specs_location = 'C:\\Users\\avitu\\Documents\\GitHub\\VQA-MED\\VQA-MED\\Cognitive-LUIS-Windows-master\\Sample\\VQA.Python\\data\\vqa_specs.pkl'

In [2]:
model_location = 'C:\\Users\\Public\\Documents\\Data\\2018\\vqa_models\\20180629_1220_23\\vqa_model_ClassifyStrategies.NLP_trained.h5'

In [3]:
# %%capture
import os
import numpy as np
from pandas import HDFStore
from vqa_logger import logger 
from enum import Enum
import IPython
from keras.models import load_model

c:\local\Anaconda3-4.1.1-Windows-x86_64\envs\conda_env\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
from common.constatns import images_path_test
from common.utils import VerboseTimer
from parsers.VQA18 import Vqa18Base
from common.functions import get_size, get_highlited_function_code, normalize_data_strucrture
from vqa_logger import logger
from common.os_utils import File 

In [5]:
with VerboseTimer("Loading Model"):
    model = load_model(model_location)

Loading Model: 0:00:02.946927


In [6]:
vqa_specs = File.load_pickle(vqa_specs_location)
data_location = vqa_specs.data_location
data_location

'C:\\Users\\avitu\\Documents\\GitHub\\VQA-MED\\VQA-MED\\Cognitive-LUIS-Windows-master\\Sample\\VQA.Python\\data\\model_input.h5'

In [7]:
code = get_highlited_function_code(normalize_data_strucrture,remove_comments=True)
IPython.display.display(code)

In [8]:
logger.debug(f"Loading test data from {data_location}")
with VerboseTimer("Loading Test Data"):
    with HDFStore(data_location) as store:        
        df_data = store['test']

[22:30:53][DEBUG] Loading test data from C:\Users\avitu\Documents\GitHub\VQA-MED\VQA-MED\Cognitive-LUIS-Windows-master\Sample\VQA.Python\data\model_input.h5
Loading Test Data: 0:00:00.673330


In [9]:
df_data.head()

,image_name,question,answer,group,path,answer_embedding,question_embedding,image
0,AIAN-16-292-g001.jpg,where is the lesion located?,NaN,test,C:\Users\Public\Documents\Data\2018\VQAMed2018...,,"[[-0.24224531650543213, 0.43569791316986084, -...","[[[28, 28, 28], [26, 26, 26], [21, 21, 21], [2..."
1,1757-1626-0002-0000007275-001.jpg,what does the abdominal ct show?,NaN,test,C:\Users\Public\Documents\Data\2018\VQAMed2018...,,"[[-1.792357087135315, 2.5371570587158203, 0.78...","[[[4, 3, 5], [5, 4, 6], [11, 10, 14], [7, 6, 1..."
2,jkms-25-1536-g002.jpg,where is the lesion located?,NaN,test,C:\Users\Public\Documents\Data\2018\VQAMed2018...,,"[[-0.24224531650543213, 0.43569791316986084, -...","[[[37, 37, 37], [37, 37, 37], [37, 37, 37], [3..."
3,IJRI-23-243-g004.jpg,what is the hypointense signal suggestive of?,NaN,test,C:\Users\Public\Documents\Data\2018\VQAMed2018...,,"[[-2.576011896133423, 1.1504747867584229, -0.0...","[[[45, 44, 46], [52, 52, 53], [54, 51, 53], [6..."
4,kjh-47-142-g001.jpg,what does the chest ct image show?,NaN,test,C:\Users\Public\Documents\Data\2018\VQAMed2018...,,"[[-1.7583001852035522, 2.477630615234375, 0.76...","[[[23, 23, 23], [20, 20, 20], [23, 23, 23], [2..."


In [11]:
#TODO: Duplicate:

In [21]:
def concate_row(df, col):
    return np.concatenate(df[col], axis=0)

def get_features_and_labels(df):
    image_features = np.asarray([np.array(im) for im in df['image']])
    # np.concatenate(image_features['question_embedding'], axis=0).shape
    question_features = concate_row(df, 'question_embedding') 

    features = ([f for f in [question_features, image_features]])
    labels = None# concate_row(df, 'answer_embedding')
    return features, labels


In [22]:
features, _ = get_features_and_labels(df_data)

In [25]:

p = model.predict(features)

In [26]:
p

array([[2.1991243e-03, 6.2448200e-04, 3.7868265e-13, ..., 2.8776132e-14,
        2.8661975e-14, 3.2802506e-14],
       [7.8926963e-04, 3.0502008e-06, 9.8802566e-08, ..., 3.1193473e-15,
        2.3324894e-15, 2.1542314e-15],
       [7.2746834e-04, 3.8213318e-05, 3.0096334e-14, ..., 6.4665396e-16,
        5.6478807e-16, 7.1179985e-16],
       ...,
       [1.4163327e-03, 1.3800043e-04, 1.1205324e-10, ..., 1.4214472e-13,
        1.0711815e-13, 1.2806141e-13],
       [2.7324797e-03, 1.2681780e-04, 4.3154436e-08, ..., 1.4278004e-14,
        1.6359449e-14, 2.2394264e-14],
       [3.2263366e-03, 1.3703281e-04, 2.9111610e-10, ..., 5.4152981e-14,
        4.1082616e-14, 4.4914010e-14]], dtype=float32)